In [1]:
import numpy as np

import cv2

import glob
import ntpath

from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
from collections import Counter

In [3]:
%matplotlib notebook

In [5]:
path = r'C:/Users/mkotyushev/Documents/z/'
apparel_type = r"outwear/"

In [ ]:
condition = [False]*5
MIN_MATCH_COUNT = 10
appropriate_scale = 3

names_coincides = []
photos_matches = []

path = r'C:/Users/mkotyushev/Documents/z_tiny_cut/'
kaze = cv2.KAZE_create()

for filename_a in glob.glob(path + "*.jpg"):
    single_name_coincides = []
    single_photo_matches = []
    
    img1 = cv2.imread(filename_a, 0)
    kp1, des1 = kaze.detectAndCompute(img1,None)
    
    for filename_b in glob.glob(path + "*.jpg"):
        if int(ntpath.basename(filename_a).split("_")[0]) == int(ntpath.basename(filename_b).split("_")[0]):
            single_name_coincides.append("1")
        else:
            single_name_coincides.append("0")
        
        
        
        img2 = cv2.imread(filename_b, 0)

        kp2, des2 = kaze.detectAndCompute(img2,None)
        bf = cv2.BFMatcher(crossCheck=True)
        matches = bf.match(des1,des2)
        matches = sorted(matches, key = lambda x:x.distance)
        if len(matches) >= MIN_MATCH_COUNT:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,3)
            matchesMask = mask.ravel().tolist()

            h,w = img1.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

            img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

            scale_x = np.sqrt(M[0,0]**2+M[1,0]**2)
            scale_y = np.sqrt(M[0,1]**2+M[1,1]**2)

            is_enough = np.sum(mask) > 8
            is_not_too_distorted = 0.5 < scale_x/scale_y < 2
            is_not_too_scaled = (1/appropriate_scale < scale_x < appropriate_scale) and (1/appropriate_scale < scale_y < appropriate_scale)

            if is_enough & is_not_too_distorted & is_not_too_scaled:
                single_photo_matches.append("1")
            else:
                single_photo_matches.append("0")
        else:
            single_photo_matches.append("0")
    names_coincides.append(single_name_coincides)
    photos_matches.append(single_photo_matches)
        

In [ ]:
unique, counts = np.unique(np.array(names_coincides, int)-np.array(photos_matches, int), return_counts=True)
errors = dict(zip(unique, counts))
unique, counts = np.unique(np.array(names_coincides, int), return_counts=True)
real_values = dict(zip(unique, counts))
unique, counts = np.unique(np.array(photos_matches, int), return_counts=True)
compute = dict(zip(unique, counts))

positive = compute[1] - 12

negative = compute[0]

false_positive =   errors[-1]

true_positive = compute[1] - errors[-1] -12

false_negative = errors[1]

true_negative = compute[0] - errors[1]

accuracy = (true_positive+true_negative)/(false_positive+false_negative+true_positive+true_negative)
precision = (true_positive)/(false_positive+true_positive)
recall = (true_positive)/(false_positive+false_negative)
f1 = 2*(precision*recall)/(precision+recall)

In [178]:
f1

0.24000000000000002

In [179]:
accuracy

0.76515151515151514

In [180]:
recall

0.19354838709677419

In [181]:
precision

0.31578947368421051

In [ ]:
f1

In [ ]:
accuracy

In [ ]:
recall

In [ ]:
precision

In [158]:
np.array(names_coincides)

array([['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
       ['1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
       ['0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0'],
       ['0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0'],
       ['0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0'],
       ['0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0'],
       ['0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0'],
       ['0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1']],
      dtype='<U1')

In [157]:
np.array(photos_matches)

array([['1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0'],
       ['0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1'],
       ['0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '0', '0'],
       ['0', '1', '0', '1', '1', '1', '0', '0', '0', '1', '0', '0'],
       ['1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '0'],
       ['0', '0', '1', '1', '0', '1', '1', '1', '0', '0', '1', '0'],
       ['1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0'],
       ['0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0'],
       ['1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0'],
       ['0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0'],
       ['0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1']],
      dtype='<U1')